In [3]:
import pandas as pd

# --- Step 1. Load and clean the data ---
df = pd.read_csv("/content/Parking_Tickets_Cville.csv")

# Clean and standardize street names
df['StreetName'] = df['StreetName'].astype(str).str.strip().str.upper()
df = df.dropna(subset=['StreetName'])

# Parse dates and extract the year (for normalization)
df['DateIssued'] = pd.to_datetime(df['DateIssued'], errors='coerce')
df['Year'] = df['DateIssued'].dt.year

# --- Step 2. Aggregate ticket frequency by street ---
street_counts = (
    df.groupby('StreetName')
      .agg(Total_Tickets=('RecordID', 'count'),
           Avg_Per_Year=('RecordID', lambda x: len(x)/df['Year'].nunique()))
      .reset_index()
)

# --- Step 3. Create a normalized "likelihood score" (0–1 scale) ---
street_counts['Likelihood_Score'] = (
    street_counts['Total_Tickets'] / street_counts['Total_Tickets'].max()
)

# --- Step 4. Define a function that estimates relative likelihood ---
def estimate_ticket_risk(address):
    """Return the likelihood score for an address based on its street name."""
    address = str(address).upper().strip()
    for street in street_counts['StreetName']:
        if street in address:
            score = street_counts.loc[
                street_counts['StreetName'] == street, 'Likelihood_Score'
            ].values[0]
            return street, round(score, 3)
    return None, None

# --- Step 5. Prompt user input ---
print("🚗 Charlottesville Parking Ticket Risk Estimator 🚓")
print("Enter an address in Charlottesville (e.g. '1700 Jefferson Park Ave' or '200 W Main St')\n")

user_address = input("Address: ")

street, score = estimate_ticket_risk(user_address)

# --- Step 6. Contextualize and display results ---
if score is not None:
    if score > 0.75:
        risk_level = "very high"
    elif score > 0.5:
        risk_level = "moderate to high"
    elif score > 0.25:
        risk_level = "moderate"
    else:
        risk_level = "low"

    print(f"\n📍 Street matched: {street.title()}")
    print(f"💡 Relative likelihood score: {score} (on a 0–1 scale)")
    print(f"This means streets like **{street.title()}** receive roughly "
          f"{score*100:.0f}% as many tickets as the most ticketed street in Charlottesville.")
    print(f"Your risk of getting a ticket there is considered **{risk_level.upper()}** compared to other areas.\n")
else:
    print("\n⚠️ Sorry — this address doesn’t appear in the dataset.")
    print("Try simplifying it (e.g., only include the street name and number). Example: '400 W Main St'")


/tmp/ipython-input-946021937.py:4: DtypeWarning: Columns (1,4,5,7,8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/Parking_Tickets_Cville.csv")


🚗 Charlottesville Parking Ticket Risk Estimator 🚓
Enter an address in Charlottesville (e.g. '1700 Jefferson Park Ave' or '200 W Main St')

Address: 1700 Jefferson Park Ave, Charlottesville, VA

📍 Street matched: Jefferson Park Av
💡 Relative likelihood score: 0.096 (on a 0–1 scale)
This means streets like **Jefferson Park Av** receive roughly 10% as many tickets as the most ticketed street in Charlottesville.
Your risk of getting a ticket there is considered **LOW** compared to other areas.

